# Customer Churm Prediction

## Importing Libraries

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import seaborn as sns

## Data Collection

In [ ]:
df = pd.read_csv('C:/Users/kumar/OneDrive/Desktop/Machine Learning Project/datasets/customer churn analysis.csv')
df.shape

## Data Preprocessing

## Training and Evaluation

## Plots and Analysis

Thanks !